In [ ]:
import pandas as pd
df_filtered = pd.read_csv("dataframe_final_exportado.csv")
df_filtered['timestamp'] = pd.to_datetime(df_filtered['periodo'], format='%Y%m')
df_grouped = df_filtered.groupby(['timestamp', 'customer_id', 'product_id'], as_index=False)['tn'].sum()
print(df_grouped.shape)



# 💬 Instalar AutoGluon si es necesario
#%pip install autogluon.timeseries
import pandas as pd

from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame

import os
print(os.getcwd())



# Agregar tn total por periodo y producto
df_monthly_product = df_grouped.groupby(['timestamp', 'product_id'], as_index=False)['tn'].sum()

print(df_monthly_product.shape)

# Agregar columna 'item_id' para AutoGluon
df_monthly_product['item_id'] = df_monthly_product['product_id']

print(df_monthly_product.shape)

# ⏰ 4. Crear TimeSeriesDataFrame
ts_data = TimeSeriesDataFrame.from_data_frame(
    df_monthly_product,
    id_column='item_id',
    timestamp_column='timestamp'
)

# Completar valores faltantes
ts_data = ts_data.fill_missing_values()

# ⚙️ 5. Definir y entrenar predictor
predictor = TimeSeriesPredictor(
    prediction_length=2,
    target='tn',
    freq='MS'  # Frecuencia mensual (Month Start), 
)

predictor.fit(ts_data, num_val_windows=2, time_limit=60*60)

# 🔮 6. Generar predicción
forecast = predictor.predict(ts_data)

# Extraer predicción media y filtrar febrero 2020
forecast_mean = forecast['mean'].reset_index()
print(forecast_mean.columns)

# Tomar solo item_id y la predicción 'mean'
resultado = forecast['mean'].reset_index()[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']

# Filtrar solo febrero 2020
resultado = forecast['mean'].reset_index()
resultado = resultado[resultado['timestamp'] == '2020-02-01']

# Renombrar columnas
resultado = resultado[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']


# 💾 7. Guardar archivo
resultado.to_csv("predicciones_febrero2020_fecha_01_07_v2.csv", index=False)
resultado.head()

Beginning AutoGluon training... Time limit = 3600s
AutoGluon will save models to '/Users/wilmerandresalarcon/Library/CloudStorage/OneDrive-Personal/MCD/2 AÑO/Labo III/Trabajo_final/AutogluonModels/ag-20250706_210847'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.5.0: Tue Apr 22 19:53:27 PDT 2025; root:xnu-11417.121.6~2/RELEASE_ARM64_T6041
CPU Count:          12
GPU Count:          0
Memory Avail:       7.35 GB / 24.00 GB (30.6%)
Disk Space Avail:   372.72 GB / 460.43 GB (80.9%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'MS',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection

/Users/wilmerandresalarcon/Library/CloudStorage/OneDrive-Personal/MCD/2 AÑO/Labo III/Trabajo_final
(22349, 3)
(22349, 4)


Provided train_data has 22375 rows (NaN fraction=0.1%), 780 time series. Median time series length is 36 (min=4, max=36). 
	Removing 75 short time series from train_data. Only series with length >= 9 will be used for training.
	After filtering, train_data has 21916 rows (NaN fraction=0.1%), 705 time series. Median time series length is 36 (min=9, max=36). 

Provided data contains following columns:
	target: 'tn'
	past_covariates:
		categorical:        []
		continuous (float): ['product_id']

To learn how to fix incorrectly inferred types, please see documentation for TimeSeriesPredictor.fit

AutoGluon will gauge predictive performance using evaluation metric: 'WQL'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2025-07-06 17:08:48
Models that will be trained: ['SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'NPTS', 'DynamicOptimizedTheta', 'AutoETS', 'Chr

Index(['item_id', 'timestamp', 'mean'], dtype='object')


,product_id,tn
1,20001,1307.914526
3,20002,1075.886811
5,20003,669.514959
7,20004,492.617254
9,20005,526.061764
